<a href="https://colab.research.google.com/github/govardhan26/tamil_asr/blob/main/asr_det.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.1 MB/s eta 0:00:00


In [3]:
from transformers import pipeline

In [4]:
transcriber = pipeline("automatic-speech-recognition")

No model was supplied, defaulted to facebook/wav2vec2-base-960h and revision 55bb623 (https://huggingface.co/facebook/wav2vec2-base-960h).
Using a pipeline without specifying a model name and revision in production is not recommended.


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
audio_file = '/content/samp.mp3'

In [6]:
z = transcriber(audio_file)
print(z)

{'text': "I ONT O SAY ALSO TAY THANKS TO EVERY ONE FOR A FOR COMING OUT TO NIGHT A MY NA SEE IF I LIVE IN ARLINGTON AND I FIRST BECAME INTERESTIN HIS OWNING FOR ITHER REALLY TWO REASONS A FIRST IS I'M AN HOW MEETING MEMBER IN ONE OF THE THINGS TOW MEETING DOES TO TAKE UPZONING ARTICLES AND I IALWAYS THOUGHT THEY WERE KIND OF INTERESTING A THE SECOND COMES FROM BEING A HOMEOTER WHO HAD TO NAVIGATE THE BUILDING FERMED PROCESS I JUST THOUGHT IT WAS A LOT WOR CUMBERSON AND LOKWOR CHALLENGING THEN A ITHER I EXPECTED IT TO BE HER I THELGHT THAT IT NEEDED TO ME SO IN TWO THOUSAND AND SIXTEEN I GOT MYSELF ON ON WORKING GROUP TOWN WORKING GROUP TO RECOTIFY OUR ZONING BYLAS SO THIS WAS MOSTLY AN EDITORIAL PLAN UP OURS ONLY BY LAW WAS WRITTEN IN NINETEEN SEVENTY FIVE AND IT'S BEEN EDITED HUNDREDS OF HUNDREDS OF TIMES OVER THE LOST FORTY FOR YEARS AND IT JUST REALLY NEEDING CONCLAING UP SO ALONG THE WAY I LEARNED A BIT ABOUT THE HISTORY OF ARLINGTON'S OWNING I'VE LEARNED O LITTLE BIT HOW OUR PAPER

In [7]:
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [21]:
audio, sr = get_audio()

In [17]:
print(audio)

[0 0 0 ... 0 0 0]


In [22]:
audio = '/content/samp_test_tam.mp3'
z1 = transcriber(audio)

In [23]:
print(z1)

{'text': 'UANACUM HL AUTUM E PRETY ETE KEEN GER'}


In [25]:
!pip install datasets
from datasets import load_dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 27.4 MB/s eta 0:00:00


In [26]:
test_dataset = load_dataset("common_voice", "ta", split="test[:2%]")

/root/.cache/huggingface/modules/datasets_modules/datasets/common_voice/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf/common_voice.py:634: FutureWarning: 
            This version of the Common Voice dataset is deprecated.
            You can download the latest one with
            >>> load_dataset("mozilla-foundation/common_voice_11_0", "en")
            
  warnings.warn(


Generating train split:   0%|          | 0/2009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1781 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1779 [00:00<?, ? examples/s]

Generating other split:   0%|          | 0/7428 [00:00<?, ? examples/s]

Generating validated split:   0%|          | 0/12652 [00:00<?, ? examples/s]

Generating invalidated split:   0%|          | 0/594 [00:00<?, ? examples/s]

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/ta/6.1.0/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf. Subsequent calls will reuse this data.


In [27]:
test_dataset

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 36
})

In [30]:
test_dataset.features

{'client_id': Value(dtype='string', id=None),
 'path': Value(dtype='string', id=None),
 'audio': Audio(sampling_rate=48000, mono=True, decode=True, id=None),
 'sentence': Value(dtype='string', id=None),
 'up_votes': Value(dtype='int64', id=None),
 'down_votes': Value(dtype='int64', id=None),
 'age': Value(dtype='string', id=None),
 'gender': Value(dtype='string', id=None),
 'accent': Value(dtype='string', id=None),
 'locale': Value(dtype='string', id=None),
 'segment': Value(dtype='string', id=None)}

In [32]:
test_dataset.features['sentence']

Value(dtype='string', id=None)

In [33]:
print(test_dataset)

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 36
})


In [34]:
test_dataset.num_rows

36

In [38]:
test_dataset.save_to_disk('/content/samp_tam')

Saving the dataset (0/1 shards):   0%|          | 0/36 [00:00<?, ? examples/s]

In [39]:
test_dataset[0]

{'client_id': 'e7f76d7dedd9e6999478ab1cec1f9fd0a4289101d39cb6281e0f7af8fd03a8054b7f9c84e5ac1fba4f1ea1daa88014e614c52b75ebb94547c09fddc0fbe9436c',
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/38a26df4350f0cbefb5a50d4018959b95a7ee56e372bfbcd98491bc12d6599c7/cv-corpus-6.1-2020-12-11/ta/clips/common_voice_ta_19083401.mp3',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/38a26df4350f0cbefb5a50d4018959b95a7ee56e372bfbcd98491bc12d6599c7/cv-corpus-6.1-2020-12-11/ta/clips/common_voice_ta_19083401.mp3',
  'array': array([0., 0., 0., ..., 0., 0., 0.]),
  'sampling_rate': 48000},
 'sentence': 'அன்றுநீர் சொன்னபடி அவ்விரண்டு மூலிகையைச்',
 'up_votes': 2,
 'down_votes': 0,
 'age': 'thirties',
 'gender': 'male',
 'accent': '',
 'locale': 'ta',
 'segment': "''"}

In [40]:
transcriber = pipeline("automatic-speech-recognition",model='Rajaram1996/wav2vec2-large-xlsr-53-tamil')

/usr/local/lib/python3.9/dist-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [41]:
audio = '/content/samp_test_tam.mp3'
z1 = transcriber(audio)
z1

{'text': 'வாணக்கம் எல்லஆரும் எப்படி இருக்கியீங்க'}